### Reading and Writing Data in Text Format

### Parsing functions in pandas

- read_csv
- read_table
- read_fdf
- read_clipboard
- read_excel
- Any many more....

*Type inference* is one of the important features, where you don't have to specify   which columns are numeric, integer, boolean or string. Handling dates and other custom types requires a bit more effort, though

In [3]:
!type cho6_ex1.csv 

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


Since this is comma-delimited, we can use ``read_csv`` to read it into a DataFrame:

In [62]:
import pandas as pd 
from pandas import Series
import sys 
import numpy as np 

In [11]:
df = pd.read_csv('ch06_ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


We would also have used ``read_table`` and specifying the delimeter: 

In [12]:
pd.read_table('ch06_ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


A file will not always have a header row. Let us consider this file :

In [13]:
!type ch06_ex2.csv 

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


To read this file we have a couple of options. We can allow pandas to assign default columns names, or can specify names yourself:

In [14]:
pd.read_csv('ch06_ex2.csv') # If we do not specify the header, then the first row will be considered as the header

,1,2,3,4,hello
0,5,6,7,8,world
1,9,10,11,12,foo


In [15]:
pd.read_csv('ch06_ex2.csv', header=None)  # If we pass the argument header = None, then pandas will name the columns

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [17]:
pd.read_csv('ch06_ex2.csv',names = ['a','b','c','d','message']) # We can pass the column names explicitly

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


Suppose we want the ``message`` column to be the index of the returned DataFrame. We can either indicate that we want the column 4 as the index or column name 'message' as the index column by passing the argument index_col = 4 or index_col = 'message' 

In [18]:
pd.read_csv('ch06_ex2.csv',names = ['a','b','c','d','message'], index_col=4) 

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In [19]:
pd.read_csv('ch06_ex2.csv',names = ['a','b','c','d','message'], index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


In the event that we want to form hierarchical index from multiple columns, we have to just pass a list of column number or names.

In [20]:
!type ch06_csv_m_index.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [22]:
parsed = pd.read_csv('ch06_csv_m_index.csv',index_col = ['key1','key2'])
parsed 

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In some cases, a table might not have a fixed delimeter, using whitespace or some other pattern to seperate fields. In these cases, we can pass a regular expression as a delimiter for ``read_table``. Let us consider a file that looks like this:

In [24]:
!type ch06_ex3.txt

            A         B         C
aaa -0.264438 -1.026059 -0.619500
bbb  0.927272  0.302904 -0.032399
ccc -0.264273 -0.386314 -0.217601
ddd -0.871858 -0.348382  1.100491


While we could some munging by hand, in this case the fields are seperated by a variable amount of whitespace. This can be expressed by regular expression ``\s+``:

In [25]:
result = pd.read_table('ch06_ex3.txt', sep = '\s+')
result 

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


Because there was one fewer column name than the number of data rows, ``read_table`` infers that the first column should be the DataFrame's index in this special case.

The parser functions may have additional arguments to help us handle the wide variety of exception file formats that occur (see the table below). For example we can skip the first, third and fourth rows of a file with ``skiprows``:

In [26]:
! type ch06_ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [27]:
pd.read_csv('ch06_ex4.csv', skiprows=[0,2,3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


### read_csv / read_table function arguments
- path
- sep or delimiter
- header
- index_col
- name
- skiprows 
- na_values
- comment
- parse_dates
- keep_date_col
- converters 
- day_first
- date_parser
- nrows
- iterator
- chunksize
- skip_footer
- verbose
- encoding
- sqeeze 
- thousands

Handling missing values is an important and frequently nuanced part of the file parsing process. Missing data is usually either not present (empty string) or marked by some ``sentinel`` value. By default, pandas uses a set of commonly ocurring sentinels, such as **NA, -1, #IND, and NULL**

In [28]:
!type ch06_ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [29]:
result = pd.read_csv('ch06_ex5.csv')

In [30]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [31]:
result.isnull() 

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


The ``na_values`` option can take either a list or set of strings to consider missing values :

In [32]:
result = pd.read_csv('ch06_ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Different NA sentinels can be specified for each column in a dict:

In [33]:
sentinels = {'message':['foo','NA'],'something':['two']}
result = pd.read_csv('ch06_ex5.csv', na_values=sentinels)
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


### Reading Text Files in Pieces

When processing very large files or figuring out the right set of arguments to correctly process a large file, we may only read a small piece of a file or iterate through smaller chunks of the file:

In [35]:
result = pd.read_csv('ch06_ex6.csv')
result 

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


If we want to read only a small number of rows (avoiding reading the entire file), we can specify that with ``nrows``

In [36]:
pd.read_csv('ch06_ex6.csv', nrows = 5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


To read out the file in pieces, specify a ``chunksize`` as a number of rows:

In [37]:
chunker = pd.read_csv('ch06_ex6.csv', chunksize=1000)
chunker 

``TextParser`` object returned by ``read_csv`` allows us to iterate over the parts of the file according to the ``chunksize``. For example , we can iterate over **ch06_ex6.csv**, ain the aggregating the value counts in the **'key'** column like :

In [48]:
chunker = pd.read_csv('ch06_ex6.csv', chunksize=1000)
tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value = 0)
    
tot[:10]

0    151.0
1    146.0
2    152.0
3    162.0
4    171.0
5    157.0
6    166.0
7    164.0
8    162.0
9    150.0
dtype: float64

``TextParser`` is also equipped with  a ``get_chunk`` method which enables us to read pieces of an arbitary size:

### Write Data Out to Text Format

Data can also be exported to delimited format. Let's consider one of the CSV files read above:

In [49]:
data = pd.read_csv('ch06_ex5.csv')
data 

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


Using DataFrame's ``to_csv`` method, we can write the data out to a comma-seperated file:

In [50]:
data.to_csv('ch06_ex5_out.csv')

In [51]:
! type ch06_ex5_out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


Ofcourse other delimiters can be used, (writing to ``sys.stdout`` so to just print the text result)

In [54]:
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


Missing values appear as empty strings in the output. We might want to denote them by some other sentinel value:

In [55]:
data.to_csv(sys.stdout, sep='|', na_rep='NULL')

|something|a|b|c|d|message
0|one|1|2|3.0|4|NULL
1|two|5|6|NULL|8|world
2|three|9|10|11.0|12|foo


With no other options specified, both the row and column labels are written. Both of these can be disabled:

In [58]:
data.to_csv(sys.stdout,index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


We can also write a subset of the columns, and in an order of our choice:

In [60]:
data.to_csv(sys.stdout,index=False,columns=['a','b','c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


Series also has a ``to_csv`` method:

In [67]:
dates = pd.date_range('1/1/2000',periods=7)
print(dates)
ts = Series(np.arange(7),index = dates)
print(ts)
ts.to_csv('ch06_tseries.csv') ## This method is deprecated in latest version of pandas

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07'],
              dtype='datetime64[ns]', freq='D')
2000-01-01    0
2000-01-02    1
2000-01-03    2
2000-01-04    3
2000-01-05    4
2000-01-06    5
2000-01-07    6
Freq: D, dtype: int32


C:\Users\Home-PC\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


### Manually Working with Delimited Formats

Most form of tabular data can be loaded from disk using functions like ``pandas.read_table``. In some cases, however, some manual processing may be necessary. It is not uncommon to receive a file with one or more malformed lines that trip up ``read_table``. To illustrate the basic tools, consider a small CSV file:

In [79]:
!type ch06_ex7.csv 

"a","b","c"
"1","2","3"
"1","2","3"


For any file with a single-character delimiter, we can use Python's built-in ``csv`` module. To use it, pass any open file or file-like object to ``csv.reader``:

In [84]:
import csv 
f = open('ch06_ex7.csv')

reader = csv.reader(f)

Iterating through the reader like a file yields a list of values in each line with any quote character removed :

In [85]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


From there it is up to us to do the wrangling necessary to put the data in the form that we need it. For example:

In [88]:
lines = list(csv.reader(open('ch06_ex7.csv')))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3']]

In [90]:
header, values = lines[0], lines[1:]
print(header)
print(values)

['a', 'b', 'c']
[['1', '2', '3'], ['1', '2', '3']]


In [91]:
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

CSV files come in many different flavors. Defining a new format with a different delimiter, string quotating, string quotation convention, or line terminator is done by defining a simple subclass of ``csv.Dialect``:

In [94]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = ';'
    quotechar = '"'

reader = csv.reader(f,dialect = my_dialect)

TypeError: "quoting" must be an integer